In [18]:
import json
import csv
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy.spatial.distance
import random
import numpy.lib.recfunctions as rfn

In [19]:
## Import data
with open('face-rating-data.json') as desc_file:
    descr = json.load(desc_file)
with open('guess-who-data.json') as data_file:    
    data = json.load(data_file)

In [20]:
#qa = np.genfromtxt("uid_qa.txt", delimiter = ",", names = True, dtype = [('int64'), ('int64'), ('int64'), ('U256'), ('U128'),])

In [21]:
#qa.dtype.names

In [22]:
fe = np.genfromtxt("uid_pre_elim.txt", delimiter = ",", names = True, dtype = [('int64'), ('int64'), ('int64'), ('U256'), ('U256'),])

In [23]:
fe.dtype.names = ('uniqueID', 'bn', 'qn', 'pre_que', 'curr_elim')

fe.dtype.names
len(fe)


3314

In [24]:
#ftd = np.genfromtxt("face_id_descr.txt", delimiter = ";", skip_header = 1 , usecols = np.arange(0,2), dtype = [('U16'), ('U2056')])

In [25]:
#ftd.dtype.names = ('img_id', 'description')

In [26]:
print(fe)

[ (34303, 1, 1, ' IMG_9048 IMG_0214 IMG_0401 IMG_7806 IMG_7942 IMG_0356 IMG_9514 IMG_9957 IMG_0566 IMG_7882 IMG_0641 IMG_9364 IMG_9634 IMG_7700 IMG_9710 IMG_0701', ' IMG_9364 IMG_7942 IMG_9710')
 (34303, 1, 2, ' IMG_9048 IMG_7806 IMG_0356 IMG_9957 IMG_0566 IMG_7882 IMG_9634 IMG_7700 IMG_0214 IMG_0401 IMG_9514 IMG_0641 IMG_0701', ' IMG_9048 IMG_0214 IMG_0401 IMG_7806 IMG_0356 IMG_9957 IMG_9514 IMG_0566 IMG_7882 IMG_0641 IMG_9634 IMG_0701')
 (34303, 2, 1, ' IMG_9876 IMG_8682 IMG_8138 IMG_7730 IMG_8592 IMG_9243 IMG_9198 IMG_8577 IMG_7547 IMG_9349 IMG_7578 IMG_9740 IMG_9861 IMG_0461 IMG_9153 IMG_7821', ' IMG_9876 IMG_9198 IMG_7730 IMG_9243 IMG_7547 IMG_9349 IMG_9740 IMG_0461')
 ...,
 (11263, 4, 1, ' IMG_9213 IMG_7730 IMG_9063 IMG_0416 IMG_0108 IMG_0199 IMG_9318 IMG_0446 IMG_7562 IMG_7988 IMG_0671 IMG_9499 IMG_0656 IMG_0153 IMG_0003 IMG_8153', ' IMG_0416 IMG_9318 IMG_7562 IMG_7988 IMG_0656 IMG_0153 IMG_8153 IMG_9213 IMG_7730 IMG_9063 IMG_0199 IMG_0671 IMG_0003')
 (11263, 4, 2, ' IMG_0446 IM

In [27]:
## first convert strings to list of rows equal to rows in fe
def mk_list(str_vector):
    ##des_byid = str_vector
    all_x = []
    for i in str_vector:
        j = i.split()
        all_x.append(j)

    return all_x


In [28]:
## all x is input of all img ids(s) before question
all_x = mk_list(fe['pre_que'])
all_cheat = mk_list(fe['curr_elim'])

In [29]:
## befor making a new column to count img ids in curr elim. 1st convert each item in that list into a list.
all_id = mk_list(fe['curr_elim'])

In [30]:
len(all_id)

3314

In [31]:
yp_all = []
for i in all_x:
    np.random.seed(len(i)*len(all_x[(all_x.index(i))-1]))
    yp_all.append(list(np.random.random_sample((len(i),))))
print(len(yp_all))

3314


In [32]:
cheat_code = zip(yp_all, all_cheat)
##for i,j in cheat_code:
##   print(i,j)

In [33]:
y_top = []
cheat_code = zip(yp_all, all_cheat)
for i,j in cheat_code:
    #print(np.array(i))
    y_top.append(list(np.array(i).argsort()[::-1][-(len(j)+1):-1]))
    #k = i.sort()
    #print(k)
print(y_top)

[[1, 11, 6], [4, 3, 1, 10, 9, 12, 5, 11, 8, 6, 7, 0], [5, 11, 13, 8, 6, 14, 7, 0], [5, 7, 1, 4, 2], [0], [0], [11, 6], [6, 4, 8, 0, 1, 3, 9, 7, 12, 5, 13, 11, 10], [9, 7, 12, 5, 13, 15, 11, 10], [0, 6, 5, 7, 1, 4, 2], [], [0], [], [8], [6, 7, 13, 2, 9, 3, 8, 1, 10, 4, 5, 0], [0], [8, 9, 2, 14, 13, 1, 11, 6], [1, 4, 2], [], [2, 4, 0, 3], [], [], [], [8, 11, 1, 12, 13, 5, 15, 9, 6, 10, 2, 3, 7, 0], [0], [13, 1, 11, 6], [11, 1, 6, 7, 10, 9], [5, 2, 4, 3], [], [], [1], [12, 8, 9, 2, 14, 13, 1, 11, 6], [2, 5, 4, 1, 0, 3], [13, 3, 6, 15], [3, 0, 8, 5, 2, 11, 1, 6, 7, 10, 9], [2, 11, 1, 6, 15, 7, 10, 9], [0, 6, 5, 7, 1, 4, 2], [12, 7, 15, 8, 10, 14, 1, 13, 4, 2, 9, 11], [2, 1, 3], [3, 9, 0, 6, 5, 7, 15, 13], [5, 7, 1, 4, 2], [2, 0], [13, 15, 8, 14, 7, 12, 9, 5, 3, 2, 11, 10], [3], [1, 2], [2, 11, 10], [3, 1, 10, 9, 12, 5, 11, 8, 6, 7, 0], [], [1], [12, 8, 9, 2, 14, 13, 1, 11, 6], [2, 5, 4, 1, 0, 3], [], [9, 6, 10, 2, 3, 7, 0], [4, 8, 5, 2, 6, 3], [1, 2], [7, 12, 9, 5, 3, 2, 11, 10], [2], [1, 

In [34]:
## zip y_top and img id
top_img = zip(y_top,all_x)
y_pred = []
for a,b in top_img:
    ##print(a)
    ##for l in a:
      ##  print(l)
    ##y_pred = []
    ##f##or a,b in arr:
    y_p = []
    for l in a:
        y_p.append(b[l])
    y_pred.append(y_p)
print(y_pred)

[['IMG_0214', 'IMG_9364', 'IMG_9514'], ['IMG_0566', 'IMG_9957', 'IMG_7806', 'IMG_9514', 'IMG_0401', 'IMG_0701', 'IMG_7882', 'IMG_0641', 'IMG_0214', 'IMG_9634', 'IMG_7700', 'IMG_9048'], ['IMG_9243', 'IMG_9740', 'IMG_0461', 'IMG_7547', 'IMG_9198', 'IMG_9153', 'IMG_8577', 'IMG_9876'], ['IMG_9861', 'IMG_7821', 'IMG_8592', 'IMG_8682', 'IMG_8577'], ['IMG_8577'], ['IMG_8577'], ['IMG_8934', 'IMG_0491'], ['IMG_9927', 'IMG_8934', 'IMG_0491', 'IMG_9439', 'IMG_9574', 'IMG_9906', 'IMG_7851', 'IMG_0581', 'IMG_0746', 'IMG_9078', 'IMG_9755', 'IMG_7866', 'IMG_9921'], ['IMG_8243', 'IMG_8964', 'IMG_9424', 'IMG_0641', 'IMG_8228', 'IMG_9379', 'IMG_0003', 'IMG_9649'], ['IMG_8889', 'IMG_9379', 'IMG_9424', 'IMG_8667', 'IMG_9529', 'IMG_0003', 'IMG_9574'], [], ['IMG_9957'], [], ['IMG_8889'], ['IMG_8622', 'IMG_9725', 'IMG_8667', 'IMG_8532', 'IMG_0641', 'IMG_8243', 'IMG_0491', 'IMG_9770', 'IMG_0476', 'IMG_8228', 'IMG_7623', 'IMG_9957'], ['IMG_8228'], ['IMG_0566', 'IMG_9409', 'IMG_7806', 'IMG_9710', 'IMG_9634', 'I

In [35]:
y_act = mk_list(fe['curr_elim'])

In [36]:
check_acc = zip(y_act, y_pred)

In [37]:
#print(list(check_acc))

In [38]:
#for a,p in check_acc:
#    print(a,p)

In [39]:
acc_all = []
for a,p in check_acc:
    #print(a,p)
    print(len(a))
    if len(a) == 0:
        acc_i = 0
    else:
        acc_i = len(set(a).intersection(set(p)))/len(set(a))
    acc_all.append(acc_i)
    

3
12
8
5
1
1
2
13
8
7
0
1
0
1
12
1
8
3
0
4
0
0
0
14
1
4
6
4
0
0
1
9
6
4
11
8
7
12
3
8
5
2
12
1
2
3
11
0
1
9
6
0
7
6
2
8
1
2
2
1
0
1
10
3
1
1
6
7
1
1
11
3
1
7
3
5
6
5
1
1
2
8
7
7
5
0
0
0
0
0
0
0
0
0
4
0
0
0
10
4
1
0
7
3
1
4
13
2
11
2
2
7
3
2
2
1
8
5
2
8
3
3
1
10
3
1
1
15
1
2
4
6
2
1
10
4
0
0
0
0
11
2
2
7
4
2
1
1
5
4
5
1
8
3
3
1
7
2
6
15
15
15
0
0
15
7
0
3
1
4
4
4
0
7
1
0
8
0
1
0
0
4
0
1
3
11
1
2
8
0
3
1
1
0
7
7
0
1
0
15
4
0
8
3
8
4
3
11
2
2
9
4
2
7
2
1
2
1
1
1
6
3
5
0
1
12
2
2
8
7
1
9
5
8
1
5
1
11
3
1
2
7
1
3
2
7
2
3
0
3
8
5
2
4
5
3
1
2
8
5
1
0
0
4
11
7
5
1
1
0
1
6
1
5
0
1
0
1
0
1
4
4
5
2
15
15
10
1
4
0
15
4
4
7
10
5
7
7
1
8
2
4
1
8
6
1
0
0
0
0
1
4
1
1
5
1
1
1
1
1
1
1
6
4
2
8
6
1
1
11
3
0
0
0
0
0
2
10
1
2
12
2
1
4
10
1
6
5
1
2
1
7
4
3
0
0
1
8
6
1
0
0
8
5
0
1
1
6
2
7
13
2
12
3
3
8
3
1
7
5
1
2
13
1
1
12
2
1
12
3
8
4
2
1
1
14
1
11
3
8
3
3
2
8
3
1
2
1
15
11
5
2
13
12
2
1
13
1
1
8
7
8
3
3
1
2
7
5
1
12
3
8
7
11
3
1
12
2
1
3
11
1
8
1
3
1
2
8
7
15
11
4
8
2
1
4
1
6
3
0
3
2
8
6
1
3
12
8
1
3
1
1
0

In [40]:
print(acc_all)

[0.3333333333333333, 0.9166666666666666, 0.75, 0.4, 0.0, 0.0, 0.0, 0.9230769230769231, 0.5, 0.8571428571428571, 0, 0.0, 0, 1.0, 0.8333333333333334, 1.0, 0.5, 0.6666666666666666, 0, 0.75, 0, 0, 0, 0.8571428571428571, 0.0, 0.25, 0.5, 0.5, 0, 0, 0.0, 0.5555555555555556, 0.8333333333333334, 0.0, 0.9090909090909091, 0.375, 0.8571428571428571, 0.75, 0.6666666666666666, 0.5, 0.8, 0.5, 0.8333333333333334, 0.0, 1.0, 0.0, 0.8181818181818182, 0, 0.0, 0.5555555555555556, 0.8333333333333334, 0, 0.2857142857142857, 0.6666666666666666, 0.5, 0.5, 0.0, 0.5, 0.5, 1.0, 0, 0.0, 0.6, 0.6666666666666666, 1.0, 1.0, 0.5, 0.7142857142857143, 0.0, 0.0, 0.7272727272727273, 0.6666666666666666, 1.0, 0.2857142857142857, 0.0, 0.8, 0.5, 0.6, 0.0, 0.0, 1.0, 0.5, 0.8571428571428571, 0.42857142857142855, 0.6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0, 0, 0.7, 0.75, 0.0, 0, 0.5714285714285714, 0.6666666666666666, 0.0, 0.75, 0.7692307692307693, 1.0, 0.7272727272727273, 0.5, 1.0, 0.7142857142857143, 0.3333333333333333, 0.5, 0.5

In [41]:
print(np.mean(acc_all))

0.436010957711
